In [1]:
from rosemary import jpt_parse_args, jpt_setup; jpt_setup()

import sys
sys.path.append('/dccstor/mit_fm/wpq/github/mitibm2023/external/open-instruct/')

/dccstor/mit_fm/wpq/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


In [2]:
import argparse
import os
import torch
import numpy as np
import pandas as pd
import time
import json
from tqdm import tqdm
import time
from eval.mmlu.categories import subcategories, categories
from eval.utils import get_next_word_predictions, load_hf_lm_and_tokenizer, query_openai_chat_model
import torch

from eval.mmlu.run_eval import format_subject, format_example, gen_prompt, eval_hf_model

choices = ["A", "B", "C", "D"]

In [3]:

parser = argparse.ArgumentParser()
parser.add_argument("--ntrain", type=int, default=5)
parser.add_argument("--data_dir", type=str, default="data/mmlu")
parser.add_argument("--save_dir", type=str, default="results/mmlu/llama-7B/")
parser.add_argument("--model_name_or_path", type=str, default=None, help="if specified, we will load the model to generate the predictions.")
parser.add_argument("--tokenizer_name_or_path", type=str, default=None, help="if specified, we will load the tokenizer from here.")
parser.add_argument("--openai_engine", type=str, default=None, help="if specified, we will use the OpenAI API to generate the predictions.")
parser.add_argument("--subjects", nargs="*", help="which subjects to evaluate. If not specified, all the 57 subjects will be evaluated.")
parser.add_argument("--n_instances", type=int, help="if specified, a maximum of n_instances per subject will be used for the evaluation.")
parser.add_argument("--eval_batch_size", type=int, default=1, help="batch size for evaluation.")
parser.add_argument("--load_in_8bit", action="store_true", help="load model in 8bit mode, which will reduce memory and speed up inference.")
parser.add_argument("--gptq", action="store_true", help="If given, we're evaluating a 4-bit quantized GPTQ model.")
parser.add_argument("--use_chat_format", action="store_true", help="If given, the prompt will be encoded as a chat format with the roles in prompt.")

model_name_or_path = '../results/baselines/huggyllama/llama-7b/'

cmd = f"""
    --ntrain 0 \
    --data_dir ../data/eval/mmlu \
    --save_dir {model_name_or_path}/eval/mmlu/ \
    --model_name_or_path {model_name_or_path} \
    --tokenizer_name_or_path {model_name_or_path} \
    --eval_batch_size 2 \
    --n_instances 10
"""

args = jpt_parse_args(parser, cmd)


In [4]:

if args.model_name_or_path:
    print("Loading model and tokenizer...")
    model, tokenizer = load_hf_lm_and_tokenizer(
        model_name_or_path=args.model_name_or_path, 
        tokenizer_name_or_path=args.tokenizer_name_or_path,
        load_in_8bit=args.load_in_8bit, 
        gptq_model=args.gptq
    )
model.device, model.dtype

Loading model and tokenizer...


Using pad_token, but it is not set yet.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(device(type='cuda', index=0), torch.bfloat16)

In [5]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [32]:

subjects = sorted(
    [
        f.split("_test.csv")[0]
        for f in os.listdir(os.path.join(args.data_dir, "test"))
        if "_test.csv" in f
    ]
)

if args.subjects:
    assert all(subj in subjects for subj in args.subjects), f"Some of the subjects you specified are not valid: {args.subjects}"
    subjects = args.subjects


if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)
if not os.path.exists(os.path.join(args.save_dir)):
    os.makedirs(os.path.join(args.save_dir))
if not os.path.exists(os.path.join(args.save_dir, "csvs")):
    os.makedirs(os.path.join(args.save_dir, "csvs"))

all_cors = []
subcat_cors = {
    subcat: [] for subcat_lists in subcategories.values() for subcat in subcat_lists
}
cat_cors = {cat: [] for cat in categories}

subjects


['abstract_algebra',
 'anatomy',
 'astronomy',
 'business_ethics',
 'clinical_knowledge',
 'college_biology',
 'college_chemistry',
 'college_computer_science',
 'college_mathematics',
 'college_medicine',
 'college_physics',
 'computer_security',
 'conceptual_physics',
 'econometrics',
 'electrical_engineering',
 'elementary_mathematics',
 'formal_logic',
 'global_facts',
 'high_school_biology',
 'high_school_chemistry',
 'high_school_computer_science',
 'high_school_european_history',
 'high_school_geography',
 'high_school_government_and_politics',
 'high_school_macroeconomics',
 'high_school_mathematics',
 'high_school_microeconomics',
 'high_school_physics',
 'high_school_psychology',
 'high_school_statistics',
 'high_school_us_history',
 'high_school_world_history',
 'human_aging',
 'human_sexuality',
 'international_law',
 'jurisprudence',
 'logical_fallacies',
 'machine_learning',
 'management',
 'marketing',
 'medical_genetics',
 'miscellaneous',
 'moral_disputes',
 'moral_sce

# function `eval_hf_model`

In [14]:
subject = subjects[0]
print(subject)

dev_df = pd.read_csv(
    os.path.join(args.data_dir, "dev", subject + "_dev.csv"), header=None
)[: args.ntrain]
test_df = pd.read_csv(
    os.path.join(args.data_dir, "test", subject + "_test.csv"), header=None
)
if args.n_instances and args.n_instances < test_df.shape[0]:
    test_df = test_df.sample(args.n_instances, random_state=42)

abstract_algebra


In [21]:

# cors, acc, probs = eval_hf_model(
#     args, subject, model, tokenizer, dev_df, test_df, args.eval_batch_size)
batch_size = args.eval_batch_size

prompts = []
for i in range(0, test_df.shape[0]):
    k = args.ntrain
    prompt_end = format_example(test_df, i, include_answer=False)
    train_prompt = gen_prompt(dev_df, subject, k)
    prompt = train_prompt + prompt_end

    tokenized_prompt = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids
    # make sure every prompt is less than 2048 tokens
    while tokenized_prompt.shape[-1] > 2048:
        k -= 1
        train_prompt = gen_prompt(dev_df, subject, k)
        prompt = train_prompt + prompt_end
        tokenized_prompt = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).input_ids

    if args.use_chat_format:
        prompt = "<|user|>\n" + prompt.strip() + "\n<|assistant|>\nThe answer is:"

    prompts.append(prompt)
prompts[0]

'The following are multiple choice questions (with answers) about  abstract algebra.\n\nStatement 1 | If a group has an element of order 10, then it has elements of orders 1, 2, and 5. Statement 2 | If a group has an element of order 2 and an element of order 3, then it has an element of order 6.\nA. True, True\nB. False, False\nC. True, False\nD. False, True\nAnswer:'

In [18]:

# get the answer for all examples
# note: here we cannot directly use convert_tokens_to_ids because the some tokenizers will automatically add space prefix.
answer_choice_ids = [tokenizer.encode(answer_choice, add_special_tokens=False)[0] for answer_choice in choices]
pred_indices, all_probs = get_next_word_predictions(
    model, tokenizer, prompts, candidate_token_ids=answer_choice_ids, return_token_predictions=False, batch_size=batch_size
)

# get the metrics
cors = []
groud_truths = test_df.iloc[:, -1].values
for i in range(len(pred_indices)):
    prediction = choices[pred_indices[i]]
    ground_truth = groud_truths[i]
    cors.append(prediction == ground_truth)

acc = np.mean(cors)
cors = np.array(cors)

all_probs = np.array(all_probs)
print("Average accuracy {:.3f} - {}".format(acc, subject))

Getting Predictions: 100%|██████████| 10/10 [00:09<00:00,  1.02it/s]

Average accuracy 0.000 - abstract_algebra


# main()

In [37]:
l = []
for subject in subjects:
    test_df = pd.read_csv(
        os.path.join(args.data_dir, "test", subject + "_test.csv"), header=None
    )
    l.append((subject, len(test_df)))
    
l.append(('avg', sum([x[1] for x in l])))
l # 14k problems.

[('abstract_algebra', 100),
 ('anatomy', 135),
 ('astronomy', 152),
 ('business_ethics', 100),
 ('clinical_knowledge', 265),
 ('college_biology', 144),
 ('college_chemistry', 100),
 ('college_computer_science', 100),
 ('college_mathematics', 100),
 ('college_medicine', 173),
 ('college_physics', 102),
 ('computer_security', 100),
 ('conceptual_physics', 235),
 ('econometrics', 114),
 ('electrical_engineering', 145),
 ('elementary_mathematics', 378),
 ('formal_logic', 126),
 ('global_facts', 100),
 ('high_school_biology', 310),
 ('high_school_chemistry', 203),
 ('high_school_computer_science', 100),
 ('high_school_european_history', 165),
 ('high_school_geography', 198),
 ('high_school_government_and_politics', 193),
 ('high_school_macroeconomics', 390),
 ('high_school_mathematics', 270),
 ('high_school_microeconomics', 238),
 ('high_school_physics', 151),
 ('high_school_psychology', 545),
 ('high_school_statistics', 216),
 ('high_school_us_history', 204),
 ('high_school_world_history',

In [34]:


for subject in tqdm(subjects, desc=f"Evaluating subjects: "):

    dev_df = pd.read_csv(
        os.path.join(args.data_dir, "dev", subject + "_dev.csv"), header=None
    )[: args.ntrain]
    test_df = pd.read_csv(
        os.path.join(args.data_dir, "test", subject + "_test.csv"), header=None
    )
    if args.n_instances and args.n_instances < test_df.shape[0]:
        test_df = test_df.sample(args.n_instances, random_state=42)

    if args.model_name_or_path:
        cors, acc, probs = eval_hf_model(args, subject, model, tokenizer, dev_df, test_df, args.eval_batch_size)
    else:
        cors, acc, probs = eval_openai_chat_engine(args, subject, args.openai_engine, dev_df, test_df, args.eval_batch_size)

    subcats = subcategories[subject]
    for subcat in subcats:
        subcat_cors[subcat].append(cors)
        for key in categories.keys():
            if subcat in categories[key]:
                cat_cors[key].append(cors)
    all_cors.append(cors)

    test_df["correct"] = cors
    for j in range(probs.shape[1]):
        choice = choices[j]
        test_df["choice{}_probs".format(choice)] = probs[:, j]
    test_df.to_csv(
        os.path.join(
            args.save_dir, 'csvs', "{}.csv".format(subject)
        ),
        index=None,
    )

for subcat in subcat_cors:
    subcat_acc = np.mean(np.concatenate(subcat_cors[subcat]))
    print("Average accuracy {:.3f} - {}".format(subcat_acc, subcat))

for cat in cat_cors:
    cat_acc = np.mean(np.concatenate(cat_cors[cat]))
    print("Average accuracy {:.3f} - {}".format(cat_acc, cat))
weighted_acc = np.mean(np.concatenate(all_cors))
print("Average accuracy: {:.3f}".format(weighted_acc))

# save results
with open(os.path.join(args.save_dir, "metrics.json"), "w") as f:
    json.dump(
        {
            "average_acc": weighted_acc,
            "subcat_acc": {
                subcat: np.mean(np.concatenate(subcat_cors[subcat]))
                for subcat in subcat_cors
            },
            "cat_acc": {
                cat: np.mean(np.concatenate(cat_cors[cat]))
                for cat in cat_cors
            },
        },
        f,
    )



Evaluating subjects:  51%|█████     | 29/57 [00:00<00:00, 159.95it/s]

100
135
152
100
265
144
100
100
100
173
102
100
235
114
145
378
126
100
310
203
100
165
198
193
390
270
238
151
545
216


Evaluating subjects:  79%|███████▉  | 45/57 [00:00<00:00, 154.69it/s]

204
237
223
131
121
108
163
112
103
234
100
783
346
895
306
311
324
282
1534


Evaluating subjects: 100%|██████████| 57/57 [00:00<00:00, 120.77it/s]

272
612
110
245
201
100
166
171
Average accuracy 0.220 - math
Average accuracy 0.275 - health
Average accuracy 0.300 - physics
Average accuracy 0.367 - business
Average accuracy 0.200 - biology
Average accuracy 0.300 - chemistry
Average accuracy 0.350 - computer science
Average accuracy 0.200 - economics
Average accuracy 0.400 - engineering
Average accuracy 0.467 - philosophy
Average accuracy 0.367 - other
Average accuracy 0.250 - history
Average accuracy 0.400 - geography
Average accuracy 0.250 - politics
Average accuracy 0.350 - psychology
Average accuracy 0.450 - culture
Average accuracy 0.233 - law
Average accuracy 0.283 - STEM
Average accuracy 0.346 - humanities
Average accuracy 0.300 - social sciences
Average accuracy 0.314 - other (business, health, misc.)
Average accuracy: 0.309
